# Assignment.
Need to find a sequence from a fasta file with specified parameters. The parameters are as follows:

Basic:
1) The sequence has two regions with a specific sequence, but mismatch (1-2 nt.) is allowed.
2) The length between the two regions is approximately 37 nt. (Internal sequence length in nt.)

Additional:
1) Sequence must be unique by NCBI Blastnt.
2) Conserved (from exons that are in all alternative splicing variants)
3) Sequence length is approximately 79 nt . (can be shorter)
4) The sequence must not form a stable secondary structure according to mfold - Quikfold (with Energy rules: RNA 4.0).

# Idea of Searching
<img src="images/process.png" height="700">

# Upload Data
Read fasta file:

In [616]:
# import needed package
from Bio import SeqIO
import pandas as pd

fasta_file = "input/NM_004448.4.exons.fa" # path to the fasta file

mRNA_seq = '' # string for forming a single sequence
with open('output/mRNA_seq.fa', 'w') as f:
        for seq_record in SeqIO.parse(fasta_file, "fasta"):
                mRNA_seq += str(seq_record.seq)

        f.write(f'> Homo sapiens erb-b2 receptor tyrosine kinase 2 (ERBB2)\n')
        f.write(f'{mRNA_seq}\n')

len_mRNA_seq = len(mRNA_seq)
print(len_mRNA_seq/10**3, 'kb', f'is a size of {seq_record.description}')

4.557 kb is a size of ref|NM_004448.4|:3588-4557 Homo sapiens erb-b2 receptor tyrosine kinase 2 (ERBB2), transcript variant 1, mRNA


# SETUP FOR SEARCHING

In [617]:
# --------SETUP FOR SEARCHING--------

# How long should the regions be that will bond with the scaffold and displace the arms?
len_seq_1 = 6 # nt. length for find_seq_1
len_seq_2 = 6 # nt. length for find_seq_2

# Internal sequence length in nt.
len_inter_seq = 37
# variance for internal sequence length in nt.
var_inter = 3

# How many mismatches are acceptable?
mm_nt_1 = 2 # number mismatch for searching sequence 1
mm_nt_2 = 2 # number mismatch for searching sequence 2

<img src="images/image_1.png" height="300">

In [618]:
# Let's define some sequences for further work
cons_seq_1 = 'TATTTTATTC' # conservative sequence 1
cons_seq_2 = 'TTGCTGTGGA' # conservative sequence 2

find_seq_1 = cons_seq_1[:len_seq_1] # seq 1 for searching process
find_seq_2 = cons_seq_2[len(cons_seq_2)-len_seq_2:] # seq 1 for searching process

# SEARCHING
For convenience of saving the found sequences, let's create a class **Marker**

In [619]:
class Marker:
        def __init__(self, region_1, region_1_cord, region_2, region_2_cord, inner_seq):
                self.region_1 = region_1
                self.region_1_cord = region_1_cord
                self.region_2 = region_2
                self.region_2_cord = region_2_cord
                self.inner_seq = inner_seq
                self.seq_marker = ' '.join([self.region_1, self.inner_seq, self.region_2])
                self.marker_cord = [region_1_cord[0]+1, region_2_cord[1]]

We need a function that will compare two sequences admitting mismatches. Let's write it

In [620]:
def compare(seq_1, seq_2, max_mm):
        """
        Сompare two sequences admitting mismatches
        :param seq_1: 1st sequence (reference)
        :param seq_2: 2nd sequence (template)
        :param max_mm: Number of acceptable mismatches
        :return: True or False
        """
        n_mm = 0
        for i in range(len(seq_1)):
                if seq_1[i] != seq_2[i]:
                        n_mm += 1
                if n_mm >= max_mm+1 or len(seq_1) != len(seq_2):
                        return False

        return True

In [621]:
list_markers = []

for i in range(len_mRNA_seq):
        current_reg_1 = mRNA_seq[i:i + len_seq_1] # get a region for further comparison

        if compare(current_reg_1, find_seq_1, mm_nt_1): # find current_reg_1 which is closer to find_seq_1
                for len_inter_j in range(len_inter_seq - var_inter, len_inter_seq + var_inter+1):
                        current_reg_2 = mRNA_seq[i + len_seq_1 + len_inter_j:i + len_seq_1 + len_inter_j + len_seq_2]
                        if len(current_reg_2) == len(find_seq_2) and compare(current_reg_2, find_seq_2, mm_nt_2):
                                reg_1 = current_reg_1
                                reg_1_cord = [i, i + len_seq_1]
                                reg_2 = current_reg_2
                                reg_2_cord = [i + len_seq_1 + len_inter_j, i + len_seq_1 + len_inter_j + len_seq_2]
                                inner_seq = mRNA_seq[reg_1_cord[1] : reg_2_cord[0]]
                                list_markers.append(Marker(reg_1, reg_1_cord, reg_2, reg_2_cord, inner_seq))


In [622]:
table_markers = pd.DataFrame(columns=['Marker', 'Length', 'Position']) # create dataframe

for marker in list_markers:
        new_row = {'Marker': marker.seq_marker,
                   'Length': len(marker.seq_marker)-2,
                   'Position': marker.marker_cord}
        new_df = pd.DataFrame([new_row])

        table_markers = pd.concat([table_markers, new_df], axis=0, ignore_index=True)

# RESULT

In [623]:
# show list of found markers
table_markers

,Marker,Length,Position
0,TGTTCT CCGATGTGTAAGGGCTCCCGCTGCTGGGGAGAGAGTTC ...,50,"[758, 807]"
1,AATTCT GCACAATGGCGCCTACTCGCTGACCCTGCAAGGGC TGGGCA,47,"[1477, 1523]"
2,TCTTTC GGAACCCGCACCAAGCTCTGCTCCACACTGCCAACCGGC...,52,"[1629, 1680]"
3,TGTTTT GGACCGGAGGCTGACCAGTGTGTGGCCTGTGCCCACTA ...,50,"[1901, 1950]"
4,TATGTC TCCCGCCTTCTGGGCATCTGCCTGACATCCACGGTGCAG...,52,"[2516, 2567]"
5,CATTCT CCGCCGGCGGTTCACCCACCAGAGTGATGTGTGGAGT T...,49,"[2854, 2902]"
6,TCTTTG TGGATTCTGAGGCCCTGCCCAATGAGACTCTAGGGTCC ...,50,"[4158, 4207]"
7,TACTTT TTTTGTTTTGTTTTTTTAAAGATGAAATAAAGACCCA G...,49,"[4425, 4473]"
8,ACTTTT TTTGTTTTGTTTTTTTAAAGATGAAATAAAGACCCA GG...,48,"[4426, 4473]"
9,CTTTTT TTGTTTTGTTTTTTTAAAGATGAAATAAAGACCCA GGGGGA,47,"[4427, 4473]"


In [624]:
# Make fasta file for NCBI blast
with open('output/list_markers.fa', 'w') as f:
        for i in range(table_markers.shape[0]):
                f.write(f'> Marker wih position {table_markers["Position"][i]}\n')
                marker_seq = table_markers["Marker"][i].replace(" ","")
                f.write(f'{marker_seq}\n')